In [3]:
# ruff: noqa: F401, E402, T201, T203, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pformat, pprint
from typing import Callable, Iterable, Iterator, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from IPython.display import HTML, clear_output, display

import arrays
import bits
import combinatorics
import graphs
import grid
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import sorting
import stack
import strings
import trees

for m in (
    arrays,
    bits,
    combinatorics,
    graphs,
    grid,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    sorting,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from mathematics import prime_numbers as primes
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

np.set_printoptions(edgeitems=5, precision=3, linewidth=300)

In [30]:
def flower_gardening(a: list, k: int, w: int) -> int:
    n = len(a)

    def can_grow(h: int) -> bool:
        aux = [0] * (n + 1)
        days = 0
        for i, e in enumerate(a):
            h += aux[i]
            diff = h - e - days
            if diff <= 0:
                continue
            days += diff
            if days > k:
                return False
            aux[min(i + w, n)] = diff
        return True

    return upper_index(can_grow, min(a), min(a) + k)

In [31]:
flower_gardening([4, 7, 6, 10, 1, 8, 1, 2], 5, 6)

5

In [32]:
flower_gardening([8, 4], 2, 2)

6

In [25]:
flower_gardening([2, 2, 2, 2, 1, 1], 2, 3)

2

In [26]:
flower_gardening([5, 8], 5, 1)

9